In [12]:
import requests
from requests.structures import CaseInsensitiveDict
import json
import numpy as np
from strava_key_gen import access_token
from typing import Dict, Tuple
import pandas as pd
import polyline
from geopy import distance
import os 

id = 5270481


# retrieve class 

def check_if_segment_in_db(id:int) -> bool:
    return str(id) in os.listdir()

def return_segment_from_api(id: int, access_token: str) -> None:
    url = f"https://www.strava.com/api/v3/segments/{id}"
    headers = CaseInsensitiveDict()
    headers["Authorization"] = f"Bearer {access_token}"
    resp = requests.get(url, headers=headers)
    if not check_if_segment_in_db(id):
        os.mkdir(str(id))
    with open(str(id) + '/strava_json.json', 'w') as f:
        json.dump(resp.json(), f)

def get_name_lat_lon_from_segment_from_json(id:int) ->Tuple[str,list]:
    f = open(str(id) + '/strava_json.json')
    segment = json.load(f)
    return segment['name'], polyline.decode(segment['map']['polyline']) 

def retrieve_segment(id: int, access_token:str) -> list:
    if not check_if_segment_in_db(id):
        return_segment_from_api(id, access_token)
    return get_name_lat_lon_from_segment_from_json(id)






def build_api_call_string(lat_lon):
    length = len(lat_lon)
    iterations =int(np.ceil(length/100))
    start, end = 0, 99
    string_lists = []
    for i in range(iterations):
        string = ''
        for j in range(start, end):
            lat, lon = lat_lon[j][0], lat_lon[j][1]
            string += str(lat) + ',' + str(lon)+ '|'
        string_lists.append(string[:-1]) # drop the last pipe
        start += 100
        
        if end + 100 > len(lat_lon):
            end = len(lat_lon) 
        else:
            end += 100
    
    return string_lists

# elevations and lat lons 
def get_elevations_lat_lon(string: str):
    url = f"https://api.opentopodata.org/v1/eudem25m?locations={string}"
    resp = requests.get(url)
    resp = resp.json()['results']
    result_list = []
    for i in range(len(resp)):
        result_list.append([resp[i]['elevation'], resp[i]['location']['lat'], resp[i]['location']['lng']])
    return result_list

def check_if_elevation_lat_lon_exists(id):
    return os.path.exists(str(id)+ '/' + 'elevation_lat_lon.npy') 


def run_api_calls(name:str, lat_lon: list) -> np.array:
    if not check_if_elevation_lat_lon_exists(id):
        print(f"getting data for {name}")
        calls = build_api_call_string(lat_lon)
        result_list = []
        for call in calls:
            result_list += get_elevations_lat_lon(call)
        ell_np=np.array([np.array(xi) for xi in result_list])
        np.save(str(id)+'/elevation_lat_lon.npy', ell_np)
        return ell_np

    ell_np = np.load(str(id)+'/elevation_lat_lon.npy')
    return ell_np



# deltas 
def check_if_rise_run_grade_exists(id):
    return os.path.exists(str(id)+ '/' + 'rise_run_grade.npy') 

def get_distance_delta(point1: list, point2: list) -> float:
    return distance.distance(point1, point2).km * 1000 # return delta in meters 

def get_elevation_delta(point1_elev, point2_elev) -> float:
    return point2_elev - point1_elev

def get_rise_run_grade(seg_data: np.array):
    if not check_if_rise_run_grade_exists(id):
        rrg_np= np.empty((0, 3), float)
        for i in range(len(seg_data)-1):
            run = get_distance_delta(seg_data[i][1:3], seg_data[i+1][1:3])
            rise = get_elevation_delta(seg_data[i][0], seg_data[i+1][0])

            rrg_np = np.append(rrg_np, np.array([[rise, run,  rise/run *100]]), axis=0)
        
        np.save(str(id)+'/rise_run_grade.npy', rrg_np)
        return rrg_np

    rrg_np = np.load(str(id)+'/rise_run_grade.npy')
    return rrg_np

In [13]:
name, lat_lon = retrieve_segment(id, access_token)
ar = run_api_calls(name, lat_lon)
b = get_rise_run_grade(ar)

getting data for Cumbre Del Sol - Full climb


In [14]:
from physics.params import rider_weight,frontal_area,coef_drag,grade,weight_else,coef_rolling_resistance,air_density,coef_drag,drive_losses
from physics.functions import get_power_given_speed, converge_on_speed_given_power

In [46]:
def get_time_to_complete_seconds(distance: float, speed: float):
    """ distance is expressed in metres, speed is expressed in kmh.
    """
    time = 3600 * (distance / 1000)/speed
    return time

    
def get_time_in_array(target_power, distance, grade):

    speed = converge_on_speed_given_power(target_power = target_power, 
        grade = grade,
        rider_weight = rider_weight,
        weight_else = weight_else,
        coef_rolling_resistance = coef_rolling_resistance,
        air_density = air_density,
        frontal_area = frontal_area,
        coef_drag = coef_drag,
        drive_losses = drive_losses) 

    return get_time_to_complete_seconds(distance, speed)
    

In [49]:
gain = b[1][0]
distance = b[1][1] 
grade = b[1][2]
target_power = 300

In [ ]:

def converge_on_speed_given_power(target_power:float, grade: float,
    rider_weight: float,
    weight_else: float,
    coef_rolling_resistance: float,
    air_density: float,
    frontal_area: float,
    coef_drag: float,
    drive_losses: float):

    lower, upper = target_power * 0.99, target_power * 1.01
    speed_min, speed_max = 1, 100
    speed = 1
    power = 1

    while not (power > lower and power < upper):

        if power <= lower:
            speed_min = speed
            speed = np.mean([speed, speed_max])
        if power >= upper:
            speed_max = speed
            speed = np.mean([speed, speed_min])

        power = get_power_given_speed(
            grade=grade,
            rider_weight=rider_weight,
            weight_else=weight_else,
            coef_rolling_resistance=coef_rolling_resistance,
            air_density=air_density,
            frontal_area=frontal_area,
            coef_drag=coef_drag,
            drive_losses=drive_losses,
            speed=speed,
        )

        if power >= lower and power <= upper:
            return speed    


In [61]:
distance, grade

(6.1904340541319565, -31.086308723727978)

In [62]:

def converge_on_speed_given_power(target_power:float, grade: float,
    rider_weight: float,
    weight_else: float,
    coef_rolling_resistance: float,
    air_density: float,
    frontal_area: float,
    coef_drag: float,
    drive_losses: float):

    lower, upper = target_power * 0.99, target_power * 1.01
    speed_min, speed_max = 1, 60
    speed = 1
    power = 1

    
    while not (power > lower and power < upper):

        print('power = ', 'speed =', speed)

        if power <= lower:
            speed_min = speed
            speed = np.mean([speed, speed_max])
        if power >= upper:
            speed_max = speed
            speed = np.mean([speed, speed_min])

        power = get_power_given_speed(
            grade=grade,
            rider_weight=rider_weight,
            weight_else=weight_else,
            coef_rolling_resistance=coef_rolling_resistance,
            air_density=air_density,
            frontal_area=frontal_area,
            coef_drag=coef_drag,
            drive_losses=drive_losses,
            speed=speed,
        )

        counter += 1
        if power >= lower and power <= upper:
            return speed    
        
        if speed > 59: 
            return speed


In [63]:
distance, grade = b[i, 1], b[i,2]
speed = converge_on_speed_given_power(target_power = target_power, 
        grade = grade,
        rider_weight = rider_weight,
        weight_else = weight_else,
        coef_rolling_resistance = coef_rolling_resistance,
        air_density = air_density,
        frontal_area = frontal_area,
        coef_drag = coef_drag,
        drive_losses = drive_losses) 

# get_time_to_complete_seconds(distance, speed)

power =  speed = 1
power =  speed = 50.5
power =  speed = 75.25
power =  speed = 87.625
power =  speed = 93.8125
power =  speed = 96.90625
power =  speed = 98.453125
power =  speed = 99.2265625
power =  speed = 99.61328125
power =  speed = 99.806640625
power =  speed = 99.9033203125
power =  speed = 99.95166015625
power =  speed = 99.975830078125
power =  speed = 99.9879150390625
power =  speed = 99.99395751953125
power =  speed = 99.99697875976562
power =  speed = 99.99848937988281
power =  speed = 99.9992446899414
power =  speed = 99.9996223449707
power =  speed = 99.99981117248535
power =  speed = 99.99990558624268
power =  speed = 99.99995279312134
power =  speed = 99.99997639656067
power =  speed = 99.99998819828033
power =  speed = 99.99999409914017
power =  speed = 99.99999704957008


In [57]:
speed

20.72265625

In [54]:
for i in range(len(b)):
    print(i)
    get_time_in_array(target_power, b[i, 1], b[i,2])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


KeyboardInterrupt: 

In [35]:
b[:, 1].shape

(135,)

In [30]:
0.000548216438553513

1.9735791787926467